In [ ]:
# !cp '/content/drive/MyDrive/Colab Notebooks/He-simple_rnn_data.zip' ./simple_rnn_data.zip
# !cp '/content/drive/MyDrive/Colab Notebooks/He-simple_rnn_utils.py' ./utils.py

In [ ]:
# !unzip simple_rnn_data.zip

In [1]:
!pip install icecream

In [1]:
from icecream import ic

In [3]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

from utils import ALL_LETTERS, N_LETTERS
from utils import load_data, letter_to_tensor, line_to_tensor, random_training_example


In [ ]:
#只有线性

class RNN(nn.Module):
    # implement RNN from scratch rather than using nn.RNN
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        # 初始化
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size) # (*, H_{in})
        self.i2o = nn.Linear(input_size + hidden_size, output_size) #定义完之后调用就行
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input_tensor, hidden_tensor):
        combined = torch.cat((input_tensor, hidden_tensor), 1)

        hidden = self.i2h(combined)
        output = self.i2o(combined) #output与hidden不同
        output = self.softmax(output)
        return output, hidden

    def init_hidden(self):
        return torch.zeros(1, self.hidden_size) #跟输入向量一样的形式

In [4]:
category_lines, all_categories = load_data()
# ic(category_lines)
# ic(all_categories)
n_categories = len(all_categories)

n_hidden = 128
rnn = RNN(N_LETTERS, n_hidden, n_categories) #输入、输出需要独热向量

# one step
input_tensor = letter_to_tensor('A')
hidden_tensor = rnn.init_hidden()

output, next_hidden = rnn(input_tensor, hidden_tensor)
# print(output.size())
# print(next_hidden.size())

# whole sequence/name
input_tensor = line_to_tensor('Albert')
hidden_tensor = rnn.init_hidden()

output, next_hidden = rnn(input_tensor[0], hidden_tensor) # 输入第一个字符


# print(output.size())
# print(next_hidden.size())

In [5]:
#
def category_from_output(output):
    category_idx = torch.argmax(output).item()
    return all_categories[category_idx]


ic(category_from_output(output))

ic| category_from_output(output): 'Korean'


'Korean'

In [6]:
criterion = nn.NLLLoss() # 分类问题的损失函数
learning_rate = 0.005
optimizer = torch.optim.SGD(rnn.parameters(), lr=learning_rate)


def train(line_tensor, category_tensor): # 一个单词
    hidden = rnn.init_hidden()

    for i in range(line_tensor.size()[0]): # 序列长度
        output, hidden = rnn(line_tensor[i], hidden)

    loss = criterion(output, category_tensor) # 最后一个字符的output为整个单词的output

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return output, loss.item() # 最后一个字符的output为整个单词的output

In [7]:
current_loss = 0
all_losses = [] # 记录一下损失，用于plot
plot_steps, print_steps = 1000, 5000
n_iters = 100000
for i in range(n_iters):
    category, line, category_tensor, line_tensor = random_training_example(category_lines, all_categories)
    # ic(category_tensor)

    output, loss = train(line_tensor, category_tensor)
    current_loss += loss

    if (i + 1) % plot_steps == 0:
        all_losses.append(current_loss / plot_steps)
        current_loss = 0

    if (i + 1) % print_steps == 0:
        guess = category_from_output(output)
        correct = "CORRECT" if guess == category else f"WRONG ({category})"
        print(f"{i + 1} {(i + 1) / n_iters * 100} {loss:.4f} {line} / {guess} {correct}")

plt.figure()
plt.plot(all_losses)
plt.show()


def predict(input_line):
    print(f"\n> {input_line}")
    with torch.no_grad():
        line_tensor = line_to_tensor(input_line)

        hidden = rnn.init_hidden()

        for i in range(line_tensor.size()[0]):
            output, hidden = rnn(line_tensor[i], hidden)

        guess = category_from_output(output)
        print(guess)


while True:
    sentence = input("Input:")
    if sentence == "quit":
        break

    predict(sentence)



ic| category_tensor: tensor([6])
ic| category_tensor: tensor([9])
ic| category_tensor: tensor([7])
ic| category_tensor: tensor([11])
ic| category_tensor: tensor([13])
ic| category_tensor: tensor([16])
ic| category_tensor: tensor([17])
ic| category_tensor: tensor([17])
ic| category_tensor: tensor([17])
ic| category_tensor: tensor([7])
ic| category_tensor: tensor([2])
ic| category_tensor: tensor([0])
ic| category_tensor: tensor([14])
ic| category_tensor: tensor([1])
ic| category_tensor: tensor([14])
ic| category_tensor: tensor([7])
ic| category_tensor: tensor([2])
ic| category_tensor: tensor([4])
ic| category_tensor: tensor([9])
ic| category_tensor: tensor([1])
ic| category_tensor: tensor([5])
ic| category_tensor: tensor([14])
ic| category_tensor: tensor([5])
ic| category_tensor: tensor([1])
ic| category_tensor: tensor([3])
ic| category_tensor: tensor([13])
ic| category_tensor: tensor([17])
ic| category_tensor: tensor([13])
ic| category_tensor: tensor([11])
ic| category_tensor: tensor([5